In [5]:
import os
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [6]:
SEASONS = list(range(2021,2024))
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [7]:
SEASONS

[2021, 2022, 2023]

In [14]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
            print(driver.title)
            html = driver.page_source
        except TimeoutException:
            print(f"Timeout error on {url}")
            continue
        else:
            driver.quit()
            break
    return html

In [15]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html, "html.parser")
    filter = soup.find("div", {"class": "filter"})
    links = filter.find_all("a")
    href = [l.get('href') for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]


    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

In [16]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com


c:\Users\huyda\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:612: RuntimeWarning: coroutine 'get_html' was never awaited
  o.setup(parent, previous_element, next_element, previous_sibling, next_sibling)


2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [17]:
standings_files = os.listdir(STANDINGS_DIR)

In [18]:
standings_files

['NBA_2016_games-april.html',
 'NBA_2016_games-december.html',
 'NBA_2016_games-february.html',
 'NBA_2016_games-january.html',
 'NBA_2016_games-june.html',
 'NBA_2016_games-march.html',
 'NBA_2016_games-may.html',
 'NBA_2016_games-november.html',
 'NBA_2016_games-october.html',
 'NBA_2017_games-april.html',
 'NBA_2017_games-december.html',
 'NBA_2017_games-february.html',
 'NBA_2017_games-january.html',
 'NBA_2017_games-june.html',
 'NBA_2017_games-march.html',
 'NBA_2017_games-may.html',
 'NBA_2017_games-november.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2018_games-december.html',
 'NBA_2018_games-february.html',
 'NBA_2018_games-january.html',
 'NBA_2018_games-june.html',
 'NBA_2018_games-march.html',
 'NBA_2018_games-may.html',
 'NBA_2018_games-november.html',
 'NBA_2018_games-october.html',
 'NBA_2019_games-april.html',
 'NBA_2019_games-december.html',
 'NBA_2019_games-february.html',
 'NBA_2019_games-january.html',
 'NBA_2019_games-june.html',
 'N

In [19]:
async def scrape_game(standings_file):
    with open(standings_file, 'r', encoding='utf-8') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+", encoding='utf-8') as f:
            f.write(html)

In [20]:
for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]

    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)

        await scrape_game(filepath)

Mavericks vs Heat, April 1, 2023 | Basketball-Reference.com
Clippers vs Pelicans, April 1, 2023 | Basketball-Reference.com
Raptors vs Hornets, April 2, 2023 | Basketball-Reference.com
Jazz vs Nets, April 2, 2023 | Basketball-Reference.com
Grizzlies vs Bulls, April 2, 2023 | Basketball-Reference.com
Trail Blazers vs Timberwolves, April 2, 2023 | Basketball-Reference.com
Mavericks vs Hawks, April 2, 2023 | Basketball-Reference.com
Wizards vs Knicks, April 2, 2023 | Basketball-Reference.com
Pistons vs Magic, April 2, 2023 | Basketball-Reference.com
Spurs vs Kings, April 2, 2023 | Basketball-Reference.com
Lakers vs Rockets, April 2, 2023 | Basketball-Reference.com
Suns vs Thunder, April 2, 2023 | Basketball-Reference.com
Pacers vs Cavaliers, April 2, 2023 | Basketball-Reference.com
76ers vs Bucks, April 2, 2023 | Basketball-Reference.com
Warriors vs Nuggets, April 2, 2023 | Basketball-Reference.com
Raptors vs Hornets, April 4, 2023 | Basketball-Reference.com
Heat vs Pistons, April 4, 2023 